In [ ]:
from orcapod.execution_engines import NativeRayAsyncEngine
import orcapod as op
import pyarrow as pa

In [ ]:
input_stream = op.streams.ImmutableTableStream(
    pa.Table.from_pylist([{"id": i, "x": i * 2, "y": i * 3} for i in range(30)]),
    tag_columns=["id"],
)

In [ ]:
ray_engine = NativeRayAsyncEngine(
    "ray://raycluster-op-test-kuberay-head-svc.ray.svc.cluster.local:10001"
)

In [ ]:
from time import sleep


@op.function_pod("sum")
def add_numbers(x: int, y: int) -> int:
    """
    A simple function that adds two numbers.
    """
    sleep(0.5)
    return x + y

Run first synchronously

In [ ]:
result_stream1 = add_numbers(input_stream)
result_stream1.run()
result_stream1.as_df()

Now let's run it asynchronously using the Ray engine

In [ ]:
result_stream2 = add_numbers(input_stream)
result_stream2.run(ray_engine)
result_stream2.as_df()

**NOTE**: Depending on the availability of nodes and how Ray was configured, you may *not* see any improvement in the running speed for the example above (it may even take longer due to overhead!). If you observe that you don't seem to be getting any speed up, please consult your Ray cluster administrator.